<a href="https://colab.research.google.com/github/ilija-ra/ElectricityConsumptionForecast.BE/blob/main/ElectricityConsumptionForecast_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [2]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import statsmodels.api as sm
from scipy.stats import norm
from scipy.stats import kurtosis
from scipy.stats import skew
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn import datasets
from sklearn.neural_network import MLPRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
# from deap import base, creator, tools, algorithms
from random import randint
import random

In [ ]:
# !zipinfo NYSLoadData.zip

In [ ]:
# !unzip NYSLoadData.zip

Archive:  NYSLoadData.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of NYSLoadData.zip or
        NYSLoadData.zip.zip, and cannot find NYSLoadData.zip.ZIP, period.


**Loading files**

In [ ]:
csv_weather_files = glob.glob('*.{}'.format('csv'))
csv_load_files = glob.glob(os.path.join('NYSLoadData', '**', '*.csv'), recursive=True)

In [ ]:
nyYears = pd.concat([pd.read_csv(f) for f in csv_weather_files ], ignore_index=True)
nyLoadYears = pd.concat([pd.read_csv(file) for file in csv_load_files], ignore_index=True)

**Load type files preprocessing**

In [ ]:
nyLoadYears.rename(columns = {'Time Stamp':'datetime'}, inplace = True)

In [ ]:
nyLoadYears = nyLoadYears[nyLoadYears['Name'] == 'N.Y.C.'];
nyLoadYears['datetime'] = pd.to_datetime(nyLoadYears['datetime'])

nyLoadYears = nyLoadYears[(nyLoadYears['datetime'].dt.minute == 0) & (nyLoadYears['datetime'].dt.second == 0)]

In [ ]:
# nyYears = pd.read_csv('NYSWeatherDataMerged.csv')
# nyLoadYears = pd.read_csv('NYSLoadDataMerged.csv')

In [ ]:
nyLoadYears.drop('Name', axis = 1, inplace= True)
nyLoadYears.drop('PTID', axis = 1, inplace= True)

In [ ]:
nyLoadYears['Load'].interpolate(inplace = True)

**Weather type files preprocessing**

In [ ]:
nyYears.info()

In [ ]:
total = nyYears.isnull().sum().sort_values(ascending=False)
percent = (nyYears.isnull().sum()/len(nyYears)).sort_values(ascending=False)
NaNs_percentage = pd.concat([total, percent*100], axis=1, keys=['Total', 'Percentage'])
NaNs_percentage

In [ ]:
precip = (nyYears['precip'] == 0).sum()/len(nyYears)
snow = (nyYears['snow'] == 0).sum()/len(nyYears)
snowdepth = (nyYears['snowdepth'] == 0).sum()/len(nyYears)
print(precip)
print(snow)
print(snowdepth)

In [ ]:
nyYears.drop('name', axis = 1, inplace= True)
nyYears.drop('preciptype', axis = 1, inplace= True)
nyYears.drop('severerisk', axis = 1, inplace= True)
nyYears.drop('precipprob', axis = 1, inplace= True)
nyYears.drop('windgust', axis = 1, inplace= True)
nyYears.drop('solarenergy', axis = 1, inplace= True)
nyYears.drop('solarradiation', axis = 1, inplace= True)
nyYears.drop('precip', axis = 1, inplace= True)
nyYears.drop('snow', axis = 1, inplace= True)
nyYears.drop('snowdepth', axis = 1, inplace= True)
nyYears

In [ ]:
nyYears['datetime'] = pd.to_datetime(nyYears['datetime'])

In [ ]:
nyYears.describe().round()

In [ ]:
print(nyYears['temp'].unique())

In [ ]:
nyYears.loc[nyYears['temp'] > 134, 'temp'] = np.nan

In [ ]:
nyYears['temp'].interpolate(inplace = True)
nyYears['feelslike'].interpolate(inplace = True)
nyYears['dew'].interpolate(inplace = True)
nyYears['humidity'].fillna(method = 'bfill', inplace = True)
nyYears['windspeed'].interpolate(inplace = True)
nyYears['winddir'].interpolate(inplace = True)
nyYears['sealevelpressure'].interpolate(inplace = True)
nyYears['cloudcover'].interpolate(inplace = True)
nyYears['visibility'].interpolate(inplace = True)

In [ ]:
nyYears.describe().round()

In [ ]:
# Clear = 0
nyYears.loc[nyYears['conditions'] == "Clear", 'conditions'] = 0
# Partially cloudy = 1
nyYears.loc[nyYears['conditions'] == "Partially cloudy", 'conditions'] = 1
# Overcast = 2
nyYears.loc[nyYears['conditions'] == "Overcast", 'conditions'] = 2
# Rain = 3
nyYears.loc[nyYears['conditions'] == "Rain", 'conditions'] = 3
# Snow = 4
nyYears.loc[nyYears['conditions'] == "Snow", 'conditions'] = 4
# Rain, Overcast = 5
nyYears.loc[nyYears['conditions'] == "Rain, Overcast", 'conditions'] = 5
# Rain, Partially cloudy = 6
nyYears.loc[nyYears['conditions'] == "Rain, Partially cloudy", 'conditions'] = 6
# Snow, Partially cloudy = 7
nyYears.loc[nyYears['conditions'] == "Snow, Partially cloudy", 'conditions'] = 7
# Snow, Overcast = 8
nyYears.loc[nyYears['conditions'] == "Snow, Overcast", 'conditions'] = 8

In [ ]:
nyYears['conditions'].fillna(method = 'ffill', inplace = True)

In [ ]:
total = nyYears.isnull().sum().sort_values(ascending=False)
percent = (nyYears.isnull().sum()/len(nyYears)).sort_values(ascending=False)
NaNs_percentage = pd.concat([total, percent*100], axis=1, keys=['Total', 'Percentage'])
NaNs_percentage

**MERGED BOTH TYPES INTO ONE DATASET - preprocessing**

In [ ]:
nyYears_merged = nyYears.merge(nyLoadYears, on = 'datetime', how = 'right')

In [ ]:
# nyYears_merged['datetime'] = nyYears_merged['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
# 2020-03-29 01:00:00
nyYears_merged['year'] = nyYears_merged['datetime'].dt.year
nyYears_merged['month'] = nyYears_merged['datetime'].dt.month
nyYears_merged['day'] = nyYears_merged['datetime'].dt.day
nyYears_merged['hour'] = nyYears_merged['datetime'].dt.hour
nyYears_merged['day_of_week'] = nyYears_merged['datetime'].dt.dayofweek + 1

In [ ]:
# # Create a new column for the mean temperature of the same day and month in previous years
# nyYears_merged['mean_temp_same_day_prev_years'] = nyYears_merged.apply(
#     lambda row: nyYears_merged[(nyYears_merged['day'] == row['day']) & (nyYears_merged['month'] == row['month']) & (nyYears_merged['datetime'].dt.year < row['datetime'].year)]['temp'].mean(),
#     axis=1
# )

In [ ]:
# # Calculate the average consumption for each day
# daily_avg_load = nyYears_merged.groupby(nyYears_merged['datetime'].dt.date)['Load'].mean()
# # Merge the daily average consumption back to the original DataFrame
# nyYears_merged = nyYears_merged.merge(daily_avg_load, left_on=nyYears_merged['datetime'].dt.date, right_index=True, suffixes=('', '_avg_prev_day'))
# nyYears_merged.drop('key_0', axis = 1, inplace= True)

# Calculate the average temperature for each day
daily_avg_temp = nyYears_merged.groupby(nyYears_merged['datetime'].dt.date)['temp'].mean()
# Merge the daily average temperature back to the original DataFrame
nyYears_merged = nyYears_merged.merge(daily_avg_temp, left_on=nyYears_merged['datetime'].dt.date, right_index=True, suffixes=('', '_avg_prev_day'))
nyYears_merged.drop('key_0', axis = 1, inplace= True)

In [ ]:
nyYears_merged.shape

In [ ]:
# EDT = 1
nyYears_merged.loc[nyYears_merged['Time Zone'] == "EDT", 'Time Zone'] = 1
# EST = 2
nyYears_merged.loc[nyYears_merged['Time Zone'] == "EST", 'Time Zone'] = 2

In [ ]:
nyYears_merged.column = nyYears_merged[['datetime', 'year', 'month', 'day', 'hour', 'day_of_week', 'temp', 'feelslike', 'dew', 'humidity', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility', 'uvindex', 'conditions', 'Time Zone', 'Load_avg_prev_day', 'temp_avg_prev_day', 'Load']]

In [ ]:
nyYears_merged = nyYears_merged.round(2)

In [ ]:
nyYears_merged.info()

In [ ]:
nyYears_merged

In [ ]:
nyYears_merged.to_csv('NYSProcessedData.csv', index=False)